In [11]:
# Impor library yang diperlukan
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# --- 1. INISIALISASI MODEL DAN MEDIAPIPE ---

# Muat model TFLite yang sudah berisi metadata
TFLITE_MODEL_PATH = 'sibi_mobilenet_metadata.tflite'
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()

# Dapatkan detail tensor input dan output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Muat file label
LABEL_PATH = 'labels.txt'
with open(LABEL_PATH, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Inisialisasi MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,                 # Fokus pada satu tangan
    min_detection_confidence=0.7,    # Tingkatkan confidence untuk hasil lebih stabil
    min_tracking_confidence=0.5
)

# --- 2. FUNGSI UNTUK MENJALANKAN DETEKSI WEBCAM ---

def run_realtime_detection():
    # Buka koneksi ke webcam (indeks 0 biasanya adalah webcam internal)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Tidak dapat membuka kamera.")
        return

    print("\nDeteksi Webcam Aktif...")
    print("Arahkan tangan Anda ke kamera.")
    print("Tekan tombol 'q' pada jendela webcam untuk keluar.")

    while cap.isOpened():
        # Baca setiap frame dari webcam
        success, frame = cap.read()
        if not success:
            print("Gagal membaca frame dari kamera.")
            break

        # Balik frame secara horizontal (efek cermin) agar lebih intuitif
        frame = cv2.flip(frame, 1)

        # Konversi warna dari BGR (OpenCV) ke RGB (MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Proses frame untuk mendeteksi tangan
        results = hands.process(rgb_frame)

        # Jika tangan terdeteksi
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Hitung kotak pembatas (bounding box) dari landmark tangan
                h, w, _ = frame.shape
                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                
                x_min, x_max = int(min(x_coords) * w), int(max(x_coords) * w)
                y_min, y_max = int(min(y_coords) * h), int(max(y_coords) * h)
                
                # Tambahkan sedikit padding
                padding = 35
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)

                # Potong gambar tangan dari frame asli
                hand_img = frame[y_min:y_max, x_min:x_max]

                if hand_img.size > 0:
                    # Preprocessing gambar tangan agar sesuai dengan input model
                    img_resized = cv2.resize(hand_img, (224, 224))
                    img_normalized = img_resized / 255.0  # Normalisasi ke [0, 1]
                    img_expanded = np.expand_dims(img_normalized, axis=0).astype(np.float32)

                    # Lakukan inferensi dengan model TFLite
                    interpreter.set_tensor(input_details[0]['index'], img_expanded)
                    interpreter.invoke()
                    output_data = interpreter.get_tensor(output_details[0]['index'])
                    
                    # Dapatkan hasil prediksi
                    predicted_index = np.argmax(output_data)
                    predicted_label = labels[predicted_index]
                    confidence = np.max(output_data)
                    
                    # Tampilkan hasil pada frame
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    label_text = f'{predicted_label} ({confidence:.2f})'
                    cv2.putText(frame, label_text, (x_min, y_min - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Tampilkan frame yang sudah diproses dalam sebuah jendela baru
        cv2.imshow('Deteksi SIBI Real-Time - Tekan Q untuk Keluar', frame)

        # Hentikan loop jika tombol 'q' ditekan
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    # Lepaskan koneksi kamera dan tutup semua jendela OpenCV
    cap.release()
    cv2.destroyAllWindows()
    print("\nDeteksi dihentikan.")

# --- 3. JALANKAN FUNGSI DETEKSI ---
run_realtime_detection()


Deteksi Webcam Aktif...
Arahkan tangan Anda ke kamera.
Tekan tombol 'q' pada jendela webcam untuk keluar.

Deteksi dihentikan.
